## Formulação matemática do problema da Grade Horária na UFPR Campus Pontal (Timetabling)


### Objetivo

O objetivo desta modelagem é minimizar a quantidade de vezes que os professores da UFPR Campus Pontal vão ao campus.
Vamos levar em conta dados originais obtidos em contato com a universidade e restrições reais.


### Parâmetros

Para modelar, temos as variáveis: 

* **T**: conjunto de turmas (8 turmas em LCE)
* **P**: conjunto de professores (16 professores em LCE)
* **H**: conjunto de horários das aulas (1 ou 2 - primeiro ou segundo horário)
* **D**: conjunto de dias da semana que ocorrem aulas (varia de segunda a sexta)

Total de combinações: $8 * 16 * 2 * 5 = 1280$.


### Variáveis de Decisão

As variáveis $x_{p,t,d,h}$ são inteiras e binárias, que definem se o professor $p$ irá (1) ou não (0) ministrar aula para a turma $t \in T$ no dia $d \in D$ e no horário $h \in H$.


### Problema de Otimização

$$
\begin{align}
    \text{minimizar \ \ \ \ \ } & \sum_{d=1}^{5}x_{p,t,d,h} \\
    \text{sujeito a \ \ \ \ \ } & \sum_{p=1}^{16}x_{p,t,d,h} \leq 1 \\
                                & \sum_{t=1}^{8}x_{p,t,d,h} \leq 1 \\
                                & \sum_{p=1}^{16}\sum_{h=1}^{2}x_{p,t,d,h} = HT_{t,d} \\
                                & \sum_{d=1}^{5}\sum_{h=1}^{2}x_{p,t,d,h} = R_{p,t} \\
                                & \sum_{t \in T_1}\sum_{p \in P_1}x_{p,t,d,h} = 2 \\
                                & \sum_{h=1}^{}x_{ptdh} \leq 6
\end{align}
$$

onde:

- $H$:
- $T_{t,d}$:
- $R_{p,t}$:
- $T_1$: subconjunto das turmas $T$ com aulas comuns às três habilitações
- $P_1$: subconjunto dos professores $P$ que ministram aulas nas disciplinas comuns

In [16]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

grade_compilada = pd.read_csv('dados/grade_compilada.csv', sep=';')
grade_compilada_tratado = pd.read_csv('dados/grade_compilada_tratado.csv', sep=',')
hora_aula_materia = pd.read_csv('dados/hora_aula_materia.csv', sep=',')
materias_comuns = pd.read_csv('dados/materias_comuns.csv', sep=',')
professores_materias = pd.read_csv('dados/professores_materias.csv', sep=',')

In [6]:
grade_compilada.head()

,Campus,Turma,Período do Dia,Parte do Horário,Semestre,Dia,Tipo de Curso,Tipo de Turma,Aula,prof,Observação
0,MIRASOL,EAQ,MANHÃ,1.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUÇÃO A AQUICULTURA,LAGREZE,NaN
1,MIRASOL,EAQ,MANHÃ,2.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUÇÃO A AQUICULTURA,LAGREZE,NaN
2,MIRASOL,EAQ,MANHÃ,3.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
3,MIRASOL,EAQ,MANHÃ,4.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
4,MIRASOL,EAQ,MANHÃ,1.0,1.0,TERÇA,MATUTINO,INTEGRAL MT,INTRODUÇÃO A QUALIDADE,SACHSIDA,NaN


In [11]:
grade_compilada_tratado.head()

,Campus,Curso,Periodo_dia,Horario,Semestre,Dia,Periodo_Aula,Tipo_Curso,Materia,Professor,Observacao
0,MIRASOL,EAQ,MANHA,1.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN
1,MIRASOL,EAQ,MANHA,2.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,INTRODUCAO A AQUICULTURA,LAGREZE,NaN
2,MIRASOL,EAQ,MANHA,3.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
3,MIRASOL,EAQ,MANHA,4.0,1.0,SEGUNDA,MATUTINO,INTEGRAL MT,PROBABILIDADE,CENDON,NaN
4,MIRASOL,EAQ,MANHA,1.0,1.0,TERCA,MATUTINO,INTEGRAL MT,INTRODUCAO A QUALIDADE,SACHSIDA,NaN


In [13]:
hora_aula_materia.head()

,Materia,Curso,Campus
0,ALGAS NOCIVAS E TOXINASOPT,EAQ,4
1,ALGEBRA LINEAR,OCEANO,3
2,ALGICULTURA,EAQ,4
3,ANALISE CEM324,LCEMATEMATICA,4
4,ANALISE COMPUTACIONAL DE ESTRUTURAS,ECV,4


In [15]:
materias_comuns.head()

,Professor,Campus,Materia
0,VALDIR,MIRASOL,PRATICA PEDAGOGICA DO ENSINO CEM335
1,JEINNI,MIRASOL,DIDATICA DAS CIENCIAS CEM334
2,JEINNI,MIRASOL,ESTAGIO CIENCIAS I CEM336
3,ELIANE,MIRASOL,DIVULGACAO CIENTIFICA CEM326


In [17]:
professores_materias.head()

,Professor,Campus,Curso,Tipo_Curso,Materia
0,LAGREZE,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A AQUICULTURA
1,CENDON,MIRASOL,EAQ,INTEGRAL MT,PROBABILIDADE
2,SACHSIDA,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A QUALIDADE
3,RODOLFO,MIRASOL,EAQ,INTEGRAL MT,INTRODUCAO A QUALIDADE
4,LUCIANA,MIRASOL,EAQ,INTEGRAL MT,GEOMETRIA ANALITICA


In [ ]:
import gurobipy as gp
from gurobipy import GRB


# Inicialização do modelo
model = gp.Model("Alocação_Professores")

# Define os conjuntos
professores = range(1, 17)
turnos = range(1, 9)
disciplinas = range(1, 6)
dias_semana = range(1, 3)

# Cria as variáveis de decisão
x = model.addVars(professores, turnos, disciplinas, dias_semana, vtype=GRB.BINARY, name="x")

# Define a função objetivo
model.setObjective(gp.quicksum(x[p,t,d,h] for p in professores for t in turnos for d in disciplinas for h in dias_semana), GRB.MINIMIZE)

# Adicionar as restrições aqui
# ...

# Resolve o modelo
model.optimize()

# Imprime a solução
if model.Status == GRB.OPTIMAL:
    for p, t, d, h in x:
        if x[p,t,d,h].X > 0:
            print(f"Professor {p} dá aula na disciplina {d} no turno {t} e no dia {h}")
else:
    print('O modelo não foi resolvido')